# Chunking and SQLite

In this notebook, we'll see a couple of techniques that can be used when working with large file in Python.

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from tqdm.notebook import tqdm

Before we do anything else, let's see how many rows are contained in the Calls for Service file. 

In [ ]:
#using terminal  "wc -l file.txt"

In [2]:
npi_cols=['NPI',
 'Entity Type Code',
 'Replacement NPI',
 'Employer Identification Number (EIN)',
 'Provider Organization Name (Legal Business Name)',
 'Provider Last Name (Legal Name)',
 'Provider First Name',
 'Provider Middle Name',
 'Provider Name Prefix Text',
 'Provider Name Suffix Text',
 'Provider Credential Text',
 'Provider First Line Business Practice Location Address',
 'Provider Second Line Business Practice Location Address',
 'Provider Business Practice Location Address City Name',
 'Provider Business Practice Location Address State Name',
 'Provider Business Practice Location Address Postal Code',
 'Provider Gender Code',
 'Healthcare Provider Taxonomy Code_1',
 'Healthcare Provider Primary Taxonomy Switch_1',
 'Healthcare Provider Taxonomy Code_2',
 'Healthcare Provider Primary Taxonomy Switch_2',
 'Healthcare Provider Taxonomy Code_3',
 'Healthcare Provider Primary Taxonomy Switch_3',
 'Healthcare Provider Taxonomy Code_4',
 'Healthcare Provider Primary Taxonomy Switch_4',
 'Healthcare Provider Taxonomy Code_5',
 'Healthcare Provider Primary Taxonomy Switch_5',
 'Healthcare Provider Taxonomy Code_6',
 'Healthcare Provider Primary Taxonomy Switch_6',
 'Healthcare Provider Taxonomy Code_7',
 'Healthcare Provider Primary Taxonomy Switch_7',
 'Healthcare Provider Taxonomy Code_8',
 'Healthcare Provider Primary Taxonomy Switch_8',
 'Healthcare Provider Taxonomy Code_9',
 'Healthcare Provider Primary Taxonomy Switch_9',
 'Healthcare Provider Taxonomy Code_10',
 'Healthcare Provider Primary Taxonomy Switch_10',
 'Healthcare Provider Taxonomy Code_11',
 'Healthcare Provider Primary Taxonomy Switch_11',
 'Healthcare Provider Taxonomy Code_12',
 'Healthcare Provider Primary Taxonomy Switch_12',
 'Healthcare Provider Taxonomy Code_13',
 'Healthcare Provider Primary Taxonomy Switch_13',
 'Healthcare Provider Taxonomy Code_14',
 'Healthcare Provider Primary Taxonomy Switch_14',
 'Healthcare Provider Taxonomy Code_15',
 'Healthcare Provider Primary Taxonomy Switch_15']

First, we need to connect to our database. The connect function will either create a new database if one does not already exist or connect to an existing one.

In [3]:
db = sqlite3.connect('nppes.sqlite')

Now, we can chunk through the data and for each row, add the rows to a table in our sqlite database. 
To keep track of how much progress has been made, we can use the `tqdm` library.

In [4]:
for chunk in tqdm(pd.read_csv('NPPES_Data_Dissemination_February_2021/npidata_pfile_20050523-20210207.csv', usecols=npi_cols, chunksize = 10000)):
    chunk.columns = [x.lower().replace(' ', '_').replace('(', '').replace(')','').replace('*','') for x in chunk.columns]      # Clean up the column names
    conditions = [
    (chunk['healthcare_provider_primary_taxonomy_switch_1']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_2']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_3']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_4']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_5']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_6']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_7']=='Y'),
    (chunk['healthcare_provider_primary_taxonomy_switch_8']=='Y')]

# create a list of the values we want to assign for each condition
    values = [chunk['healthcare_provider_taxonomy_code_1'],
          chunk['healthcare_provider_taxonomy_code_2'],
          chunk['healthcare_provider_taxonomy_code_3'],
          chunk['healthcare_provider_taxonomy_code_4'],
          chunk['healthcare_provider_taxonomy_code_5'],
          chunk['healthcare_provider_taxonomy_code_6'],
          chunk['healthcare_provider_taxonomy_code_7'],
          chunk['healthcare_provider_taxonomy_code_8']]

# create a new column and use np.select to assign values to it using our lists as arguments
    chunk['taxonomy'] = np.select(conditions, values)
    chunk=chunk[chunk.columns.drop(list(chunk.filter(regex='healthcare_provider_')))]
    chunk.to_sql('npi_data', db, if_exists = 'append', index = False)

#figure out this later
#def find_primary_code(row):
    #col = 'Healthcare Provider Primary Taxonomy Switch_'
    #for i in range(1, 16):
        #if row[col + str(i)] == 'Y':
            #return row['Healthcare Provider Taxonomy Code_' + str(i)]

/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (103,106) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (91,94,95,98,99,102,103,106) have mixed types.S

/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (32,75,78,79,82,83,86,87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (32,79,82,83,86,87,90,91,94,95,98,99,102) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)
/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (67,70,71,74,75,78,79,82,83,86,87,9

/Users/turik1/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3341: DtypeWarning: Columns (87,90,91,94,95,98) have mixed types.Specify dtype option on import or set low_memory=False.
  await eval(code_obj, self.user_global_ns, self.user_ns)


In [5]:
db.close()

In [26]:
db = sqlite3.connect('nppes.sqlite')
query='''
select *
from npi_data;
'''
npi_data = pd.read_sql(query, db)
npi_data

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,provider_gender_code,taxonomy
0,1679576722,1.0,NaN,None,None,WIEBE,DAVID,A,None,None,M.D.,3500 CENTRAL AVE,None,KEARNEY,NE,6.88473e+08,M,207X00000X
1,1588667638,1.0,NaN,None,None,PILCHER,WILLIAM,C,DR.,None,MD,1824 KING STREET,SUITE 300,JACKSONVILLE,FL,3.22045e+08,M,207RC0000X
2,1497758544,2.0,NaN,<UNAVAIL>,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",None,None,None,None,None,None,3418 VILLAGE DR,None,FAYETTEVILLE,NC,2.83045e+08,None,251G00000X
3,1306849450,1.0,NaN,None,None,SMITSON,HAROLD,LEROY,DR.,II,M.D.,810 LUCAS DR,None,ATHENS,TX,7.57513e+08,M,2085R0202X
4,1215930367,1.0,NaN,None,None,GRESSOT,LAURENT,None,DR.,None,M.D.,17323 RED OAK DR,None,HOUSTON,TX,7.70901e+08,M,207RH0003X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6714033,1326630724,1.0,NaN,None,None,DELMONTE,ROXANA,None,None,None,MSN-APRN-FNP-C,8895 FONTAINEBLEAU BLVD APT 309,None,MIAMI,FL,3.31724e+08,F,363LF0000X
6714034,1912599325,1.0,NaN,None,None,KATEHIS,GIORGIOS,None,None,None,PHARMD,391 W MAIN ST,None,HUNTINGTON,NY,1.17433e+08,M,183500000X
6714035,1821680232,1.0,NaN,None,None,NGANDJO YEPNDJOUO,SARIETTE,None,MS.,None,None,750 S STATE ST,None,ELGIN,IL,6.01238e+08,F,163WP0808X
6714036,1730771148,1.0,NaN,None,None,JONES,JEANETTE,MARIE,None,None,None,412 1ST ST,None,MIDDLESEX,NJ,8.84615e+07,F,390200000X


In [25]:
npi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 18 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   npi                                                      25 non-null     int64  
 1   entity_type_code                                         25 non-null     float64
 2   replacement_npi                                          0 non-null      object 
 3   employer_identification_number_ein                       4 non-null      object 
 4   provider_organization_name_legal_business_name           4 non-null      object 
 5   provider_last_name_legal_name                            21 non-null     object 
 6   provider_first_name                                      21 non-null     object 
 7   provider_middle_name                                     17 non-null     object 
 8   provider_name_prefix_text       

In [14]:
db = sqlite3.connect('nppes.sqlite')
nucc_tax=pd.read_csv('NPPES_Data_Dissemination_February_2021/nucc_taxonomy_210.csv')
nucc_tax.columns = [x.lower().replace(' ', '_') for x in nucc_tax.columns]
nucc_tax.to_sql('nucc_tax', db,  if_exists='replace', index = False)    


In [15]:
query='''
select *
from nucc_tax
limit 5;
'''
nucc_tax = pd.read_sql(query, db)
nucc_tax

,code,grouping,classification,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name
0,193200000X,Group,Multi-Specialty,None,A business group of one or more individual pra...,10/1/2003,None,None,[7/1/2003: new],Multi-Specialty Group
1,193400000X,Group,Single Specialty,None,A business group of one or more individual pra...,10/1/2003,None,None,[7/1/2003: new],Single Specialty Group
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,None,An allergist-immunologist is trained in evalua...,4/1/2003,None,7/1/2007,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,Definition to come...,4/1/2003,None,None,None,Allergy Physician
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,Definition to come...,4/1/2003,None,None,None,Clinical & Laboratory Immunology (Allergy & Im...


In [140]:
db.close()

In [24]:
nucc_tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   code                5 non-null      object
 1   grouping            5 non-null      object
 2   classification      5 non-null      object
 3   specialization      2 non-null      object
 4   definition          5 non-null      object
 5   effective_date      5 non-null      object
 6   deactivation_date   0 non-null      object
 7   last_modified_date  1 non-null      object
 8   notes               3 non-null      object
 9   display_name        5 non-null      object
dtypes: object(10)
memory usage: 528.0+ bytes


To speed up queries which use a specific column, we can create an **index** on that column. This causes the database to store that column in a way that helps it to retrieve rows quicker.

In [9]:
db = sqlite3.connect('nppes.sqlite')
db.execute('CREATE INDEX taxonomy ON npi_data(taxonomy)')
db.execute('CREATE INDEX code ON nucc_tax(code)')

In [11]:
db = sqlite3.connect('nppes.sqlite')
cbsa=pd.read_csv('NPPES_Data_Dissemination_February_2021/ZIP_CBSA_122020.csv')
cbsa.columns = [x.lower().replace(' ', '_') for x in cbsa.columns]
cbsa.to_sql('cbsa', db,  if_exists='replace', index = False)

In [12]:
cbsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47424 entries, 0 to 47423
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   zip        47424 non-null  int64  
 1   cbsa       47424 non-null  int64  
 2   res_ratio  47424 non-null  float64
 3   bus_ratio  47424 non-null  float64
 4   oth_ratio  47424 non-null  float64
 5   tot_ratio  47424 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 2.2 MB


In [84]:
db = sqlite3.connect('nppes.sqlite')
query= '''
select *      
from npi_data as np
left join nucc_tax as nu
on np.taxonomy=nu.code
inner join 
(select zip, cbsa, max(tot_ratio) as max_ratio
from cbsa
group by zip, cbsa) as c
on np.provider_business_practice_location_address_postal_code=c.zip 
'''
npp_nucc_tax = pd.read_sql(query, db)
npp_nucc_tax.head(25)

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,specialization,definition,effective_date,deactivation_date,last_modified_date,notes,display_name,zip,cbsa,max_ratio
0,1396748448,1.0,NaN,None,None,YOUNG-MAYKA,CYNTHIA,J,MS.,None,...,None,A physician assistant is a person who has succ...,4/1/2002,None,None,None,Physician Assistant,13069,45060,1.000000
1,1669475612,1.0,NaN,None,None,TROTCHIE,DEBBIE,C,None,None,...,Women's Health,Definition to come...,4/1/2002,None,None,None,Women's Health Nurse Practitioner,89148,29820,1.000000
2,1043213093,2.0,NaN,<UNAVAIL>,HOME I V SPECIALISTS INC,None,None,None,None,None,...,Home Infusion Therapy Pharmacy,"Pharmacy-based, decentralized patient care org...",4/1/2006,None,None,Source: National Home Infusion Association [1...,Home Infusion Therapy Pharmacy,72401,27860,0.993251
3,1043213093,2.0,NaN,<UNAVAIL>,HOME I V SPECIALISTS INC,None,None,None,None,None,...,Home Infusion Therapy Pharmacy,"Pharmacy-based, decentralized patient care org...",4/1/2006,None,None,Source: National Home Infusion Association [1...,Home Infusion Therapy Pharmacy,72401,37500,0.006749
4,1194728139,1.0,NaN,None,None,STEWART,SUSAN,S,MRS.,None,...,Medical,Definition to come...,4/1/2002,None,None,None,Medical Physician Assistant,96793,27980,1.000000
5,1396748323,1.0,NaN,None,None,CAMPBELL,LAURA,IVEY,DR.,None,...,None,An individual licensed by the appropriate stat...,4/1/2002,None,1/1/2006,Source: Adapted from National Association of B...,Pharmacist,37711,24620,0.338293
6,1396748323,1.0,NaN,None,None,CAMPBELL,LAURA,IVEY,DR.,None,...,None,An individual licensed by the appropriate stat...,4/1/2002,None,1/1/2006,Source: Adapted from National Association of B...,Pharmacist,37711,28700,0.643996
7,1396748323,1.0,NaN,None,None,CAMPBELL,LAURA,IVEY,DR.,None,...,None,An individual licensed by the appropriate stat...,4/1/2002,None,1/1/2006,Source: Adapted from National Association of B...,Pharmacist,37711,34100,0.017712
8,1255334280,1.0,NaN,None,None,HENSARLING,JAMES,KENNETH,DR.,None,...,Rheumatology,"An internist who treats diseases of joints, mu...",4/1/2003,None,7/1/2011,"Source: American Board of Medical Specialties,...",Rheumatology Physician,39232,27140,1.000000
9,1962405019,1.0,NaN,None,None,HAMAMI,ANWAR,None,None,None,...,None,An emergency physician focuses on the immediat...,4/1/2003,None,7/1/2011,"Source: American Board of Medical Specialties,...",Emergency Medicine Physician,44236,10420,1.000000


In [98]:
df=npp_nucc_tax[['npi', 'provider_business_practice_location_address_postal_code', 'zip','cbsa']]
df.sort_values(by=['cbsa'])

,npi,provider_business_practice_location_address_postal_code,zip,cbsa
188345,1013097880,57401.0,57401,10100
183907,1811085970,57401.0,57401,10100
737433,1851934335,57428.0,57428,10100
177283,1942395835,57428.0,57428,10100
252338,1013062280,57428.0,57428,10100
...,...,...,...,...
291016,1295857225,41240.0,41240,99999
291021,1023130069,59865.0,59865,99999
291023,1134241177,74301.0,74301,99999
291055,1548382500,40489.0,40489,99999


In [119]:
cbsa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47424 entries, 0 to 47423
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   zip        47424 non-null  int64  
 1   cbsa       47424 non-null  int64  
 2   res_ratio  47424 non-null  float64
 3   bus_ratio  47424 non-null  float64
 4   oth_ratio  47424 non-null  float64
 5   tot_ratio  47424 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 2.2 MB


In [86]:
df.duplicated(subset=['npi'], ).value_counts()

False    684871
True      79528
dtype: int64

In [ ]:
npp_nucc=pd.merge(npi_data, nucc_tax,left_on='taxonomy', right_on='code', how='left')

In [132]:
npp_nucc['postal_code']=npp_nucc['provider_business_practice_location_address_postal_code'].astype(str).str[0:5]
cbsa['zip1']=cbsa['zip'].astype(str).str[0:5]

In [133]:
npp_nucc_cbsa=pd.merge(npp_nucc, cbsa,left_on= 'postal_code', right_on='zip1', how='left')

In [136]:
npp_nucc_cbsa_nash=npp_nucc_cbsa[npp_nucc_cbsa['cbsa']==34980]


In [154]:
npp_nucc_cbsa_nash.to_csv('npp_taxo_cbsa_nash.csv',index=False)

In [138]:
npp_nucc_cbsa_nash.duplicated(subset=['npi'], ).value_counts()

False    41181
dtype: int64

In [179]:
npp_nucc_cbsa_nash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41181 entries, 54 to 7402091
Data columns (total 36 columns):
 #   Column                                                   Non-Null Count  Dtype  
---  ------                                                   --------------  -----  
 0   npi                                                      41181 non-null  int64  
 1   entity_type_code                                         41181 non-null  float64
 2   replacement_npi                                          0 non-null      float64
 3   employer_identification_number_ein                       8308 non-null   object 
 4   provider_organization_name_legal_business_name           8308 non-null   object 
 5   provider_last_name_legal_name                            32872 non-null  object 
 6   provider_first_name                                      32873 non-null  object 
 7   provider_middle_name                                     21229 non-null  object 
 8   provider_name_prefix_te

In [146]:
pd.read_csv('DocGraph_Hop_Teaming_2017_Non_Commercial/DocGraph_Hop_Teaming_2017.csv', nrows = 1000).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   from_npi           1000 non-null   int64  
 1   to_npi             1000 non-null   int64  
 2   patient_count      1000 non-null   int64  
 3   transaction_count  1000 non-null   int64  
 4   average_day_wait   1000 non-null   float64
 5   std_day_wait       1000 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 47.0 KB


In [ ]:
db = sqlite3.connect('nppes.sqlite')
for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2017_Non_Commercial/DocGraph_Hop_Teaming_2017.csv', chunksize = 10000)):
    chunk.to_sql('hop', db, if_exists = 'replace', index = False)


In [153]:
db = sqlite3.connect('nppes.sqlite')
query='''
select *
from hop;
'''
hop_team = pd.read_sql(query, db)
hop_team 

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1235127499,1255364378,209,364,8.602,30.989
1,1235378001,1255364378,853,1018,6.469,26.702
2,1255437109,1255364378,103,108,40.343,72.147
3,1255309605,1255364378,59,59,41.017,78.284
4,1255311338,1255364378,97,101,13.861,41.004
...,...,...,...,...,...,...
124,1235156159,1255367082,82,88,19.045,48.046
125,1235105081,1255367082,56,71,47.803,56.388
126,1235121047,1255367082,37,55,35.782,39.597
127,1225292865,1255367124,37,55,29.964,37.716


In [163]:
hop_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   from_npi           129 non-null    int64  
 1   to_npi             129 non-null    int64  
 2   patient_count      129 non-null    int64  
 3   transaction_count  129 non-null    int64  
 4   average_day_wait   129 non-null    float64
 5   std_day_wait       129 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 6.2 KB


In [180]:
npp_nucc_cbsa_nash_hop=pd.merge(npp_nucc_cbsa_nash, hop_team, left_on= 'npi', right_on='to_npi', how='inner')

In [181]:
npp_nucc_cbsa_nash_hop=pd.merge(npp_nucc_cbsa_nash_hop, hop_team, left_on= 'npi', right_on='from_npi', how='inner')

In [182]:
npp_nucc_cbsa_nash_hop.head(50)

,npi,entity_type_code,replacement_npi,employer_identification_number_ein,provider_organization_name_legal_business_name,provider_last_name_legal_name,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,...,patient_count_x,transaction_count_x,average_day_wait_x,std_day_wait_x,from_npi_y,to_npi_y,patient_count_y,transaction_count_y,average_day_wait_y,std_day_wait_y
